# trajectory_compare_buoy

compare wave buoy path data with what parcels says

needs to run an entirely new parcels simulation such that the delta times match with the wave buoy's (or at least be close enough to it)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
from parcels import FieldSet
import xarray as xr

import utils
from parcels_utils import buoycsv_to_particleds, HFRGrid
from parcels_sim import simulation, generate_sim_gif
from plot_utils import plot_trajectories, plot_points_fieldset
from thredds_utils import get_thredds_dataset, thredds_data

In [ ]:
csv_path = utils.WAVEBUOY_DATA_DIR / "wavebuoy_704-02.csv"
dataset = buoycsv_to_particleds(csv_path)
dataset

In [ ]:
filename = os.path.basename(csv_path)
nc_path = utils.WAVEBUOY_DATA_DIR / f"{os.path.splitext(filename)[0]}.nc"
dataset.to_netcdf(nc_path)
plot_trajectories([nc_path])

In [ ]:
buoy_timestamps = dataset["time"].values[0].astype("datetime64[s]")
np.diff(buoy_timestamps)

In [ ]:
padding = 0.08
buoy_lat_rng = (dataset["lat"].values[0].min() - padding, dataset["lat"].values[0].max() + padding)
buoy_lon_rng = (dataset["lon"].values[0].min() - padding, dataset["lon"].values[0].max() + padding)

In [ ]:
times = dataset.isel(traj=0)["time"].values
start = buoy_timestamps[0]
end = buoy_timestamps[-1]
thredds_dataset = get_thredds_dataset("buoy", 6, (start, end), buoy_lat_rng, buoy_lon_rng, expand_coords=True)
thredds_dataset

In [ ]:
grid = HFRGrid(thredds_dataset.sel(time=slice(buoy_timestamps[0], buoy_timestamps[10])), 6)

In [ ]:
domain = grid.get_domain()
domain

In [ ]:
domain["W"] = -118.3
domain["E"] = -117
domain["S"] = 32

In [ ]:
start_offset = 95
b = 200
plot_points_fieldset(dataset["lat"].values[0][a:b], dataset["lon"].values[0][a:b], 0, grid, domain=domain, line=True)

In [ ]:
# find a starting point of the buoy actually in the vector field
start_pos = None
start_time = None
start_time_idx = None
end_time = None
end_time_idx = None
for i in range(start_offset, dataset.dims["obs"]):
    time = dataset["time"].values[0, i]
    lat = dataset["lat"].values[0, i]
    lon = dataset["lon"].values[0, i]
    u, v = grid.get_closest_current(time, lat, lon)
    if np.isnan(u) or np.isnan(v) or abs(u) + abs(v) == 0:
        if start_pos is not None:
            end_time_idx = i
            end_time = np.datetime64(time, "h") + np.timedelta64(1, "h")
            break
    elif start_pos is None:
        start_time_idx = i
        start_pos = [lat, lon]
        start_time = np.datetime64(time, "h") - np.timedelta64(1, "h")
start_time_idx, end_time_idx, start_time, end_time, start_pos

In [ ]:
selected_ds = dataset.sel(obs=slice(start_time_idx, end_time_idx))
nc_path_selected = utils.WAVEBUOY_DATA_DIR / f"{os.path.splitext(filename)[0]}_selected.nc"
selected_ds.to_netcdf(nc_path_selected)
plot_points_fieldset(selected_ds["lat"].values[0], selected_ds["lon"].values[0], 0, grid, domain=domain, line=True)

In [ ]:
parcels_cfg = {
    "time_range": [start_time, end_time],
    "repeat_dt": -1,
    "particles_per_dt": -1,
    "max_variation": 0.01,
    "spawn_points": [
        start_pos
    ],
    "random_spawn": False,
    "simulation_dt": 300,
    "snapshot_interval": 10800,
    "save_snapshots": True,
    "shown_domain": domain
}

In [ ]:
grid = HFRGrid(thredds_dataset.sel(time=slice(start_time, end_time)), 6)
pfile_path, snap_path = simulation("test", grid, parcels_cfg)

In [ ]:
generate_sim_gif(
    "buoy_sim",
    snap_path,
    utils.PICUTRE_DIR / f"{utils.filename_dict[grid.resolution]}/partsim_test.gif",
    25
)

In [ ]:
plot_path = utils.create_path(utils.PICUTRE_DIR / "buoy")
plot_trajectories([nc_path_selected, pfile_path], legend=True, savefile=plot_path / f"buoy_comp.png")